# Face Detection and Training using face_recognition

## Required Libraries

In [2]:
import cv2
import os
import numpy as np
from PIL import Image
import pickle
from imutils import paths
import face_recognition
import time

## About The Code

First we initialize some variables,
```
image_dir = "C:/Users/Vansh/Desktop/New folder"
```
<b>image_dir</b> points to a location/directory where folders are stored with the label(person's name) as the folder's name and the folders containing corresponding person's images.
```
current_id = 0
label_ids = {}
y_labels = [] 
x_train = [] 
```

<b>current_id</b> is a unique id that we are going to assign to each label. <b>y_labels and x_trains</b> would contain the labels and encodings to be trained respectively


Next we access these directories and corresponding files(images) using <b>os.walk()</b>. <b>for file in files:</b> enables us to work on every individual image

Then, we assign the whole path to the file to the 'path' variable. Set label variable as the name of the corresponding directory the file exists in(person's name)and if the label does not exist in label_ids dictionary we add to the dictionary <i>'name' : id</i>

## Now to the main part

We read the image into a variable using the opencv library and then convert the image into RGB format.
(Optional) Next we resize the image to <i>1/4th of the original one</i> This is done just to ensure a decent training speed.
```
rgb = cv2.resize(rgb,None,fx=0.25,fy=0.25) 
```
The if condition following the above line is just to ensure that an image exists and is of correct shape. Next we convert the image into a numpy array.

### Where the magic happens

 ```
 boxes = face_recognition.face_locations(rgb,model="cnn") 
 ```
 The above returns a list of tuples containing all location/coordinates of all the faces in an image
 Generally, the 'cnn' model is advised to be used with GPUs but if your dataset is not in tens/hundreds of thousands
 or millions it works decently as long as you have resized your image like we did earlier.
 
 However, if training time is a concern change cnn to hog
 ```
 model = 'hog'
 ```
 Even though CNN is more accurate than hog it still performs really well.
 
 Next we take these boxes(coordinates) and encode them into a 128D face encoding using a ResNet model(a modified version with 27 layers and half the number of filters) 
```
encodings = face_recognition.face_encodings(rgb, boxes,10)
```
The third parameter is an optional one and holds the number of times to resample for every encoding(higher the number slower the process). default is 1. Personally I didn't find a significant difference in the results while the model was <b>cnn</b> however the results improved significantly when the model used was <b>hog</b>.


Finally we append all the encodings and corresponding labels into a respective lists
and then push these lists into a pickle file in a dictionary format where 'encodings' key holds all the encodings
and 'names' key holds the corresponding labels

### That's it folks! You have a trained face recognition model


In [3]:
start = time.time()
image_dir = "C:/Users/Vansh/Desktop/New folder" # dir where images are stored in labeled folders


current_id = 0
label_ids = {}
y_labels = [] # labels to be trained
x_train = []  # encodings to be trained

for root, dirs, files in os.walk(image_dir): 
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file) # path of image
            label = os.path.basename(root).replace(" ", "-").lower() # name of person in the image(from the folder name)
            
            
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
            print(label_ids)
            
            image = cv2.imread(path) # reading the image
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convering to RGB
            #size = (640,480)
            #rgb = rgb.resize(480,640, Image.ANTIALIAS)
            rgb = cv2.resize(rgb,None,fx=0.25,fy=0.25) 
            if rgb is not None and rgb.shape[0] != 0 and rgb.shape[1] !=0 : # checking if image exists
                image_array = np.array(rgb, "uint8")
                # converting the image into an numpy array
                
                
                boxes = face_recognition.face_locations(rgb,model="cnn") 
                # try it with model='hog' as well
                # boxes contains the coordinates of the face(s) in the frame
                # boxes = list of (y,x+w,y+h,x) or (top,right,bottom,left)
    
            # compute the facial embedding for the face
                encodings = face_recognition.face_encodings(rgb, boxes,10)
                # encoding contains A list of 128-dimensional face encodings (one for each face in the image)
                # encoding is essentially a list of 128 values representing the 128 hog features of the face

                # loop over the encodings
                for encoding in encodings:
                # add each encoding + name to our set of known names and
                # encodings
                    x_train.append(encoding)
                    y_labels.append(label)
                    
data = {"encodings": x_train, "names": y_labels}
with open("face-labels.pickle", 'wb') as f:
    pickle.dump(data, f)
    #storing trained data into a pickle file
print((time.time() - start)/60)                             

{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita': 1, 'neha': 2}
{'kartikey': 0, 'kavita':

{'kartikey': 0, 'kavita': 1, 'neha': 2, 'pankaj': 3, 'prakash': 4, 'prerna': 5, 'priyanka': 6, 'pulvin': 7, 'rachit': 8, 'rohit': 9, 'shruti': 10}
{'kartikey': 0, 'kavita': 1, 'neha': 2, 'pankaj': 3, 'prakash': 4, 'prerna': 5, 'priyanka': 6, 'pulvin': 7, 'rachit': 8, 'rohit': 9, 'shruti': 10}
{'kartikey': 0, 'kavita': 1, 'neha': 2, 'pankaj': 3, 'prakash': 4, 'prerna': 5, 'priyanka': 6, 'pulvin': 7, 'rachit': 8, 'rohit': 9, 'shruti': 10}
{'kartikey': 0, 'kavita': 1, 'neha': 2, 'pankaj': 3, 'prakash': 4, 'prerna': 5, 'priyanka': 6, 'pulvin': 7, 'rachit': 8, 'rohit': 9, 'shruti': 10}
{'kartikey': 0, 'kavita': 1, 'neha': 2, 'pankaj': 3, 'prakash': 4, 'prerna': 5, 'priyanka': 6, 'pulvin': 7, 'rachit': 8, 'rohit': 9, 'shruti': 10}
{'kartikey': 0, 'kavita': 1, 'neha': 2, 'pankaj': 3, 'prakash': 4, 'prerna': 5, 'priyanka': 6, 'pulvin': 7, 'rachit': 8, 'rohit': 9, 'shruti': 10}
{'kartikey': 0, 'kavita': 1, 'neha': 2, 'pankaj': 3, 'prakash': 4, 'prerna': 5, 'priyanka': 6, 'pulvin': 7, 'rachit': 